<a href="https://colab.research.google.com/github/rdancer/ai-voice-call-demo/blob/master/ai_voice_call_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
# print cell execution time for every cell
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.0 MB/s eta 0:00:00
time: 292 µs (started: 2023-11-30 08:49:56 +00:00)


In [ ]:
!pip install git+https://github.com/twilio/twilio-python

  Cloning https://github.com/twilio/twilio-python to /tmp/pip-req-build-sfyxm9k5
  Running command git clone --filter=blob:none --quiet https://github.com/twilio/twilio-python /tmp/pip-req-build-sfyxm9k5
  Resolved https://github.com/twilio/twilio-python to commit 33363be292437ae10b71be1d77d7006940e5b08b
  Preparing metadata (setup.py) ... done
  Created wheel for twilio: filename=twilio-8.10.2-py2.py3-none-any.whl size=1761893 sha256=59efbf38d6759b4f7d9c3ae18a4bdd19c709d05c21ee5c9c8172611b177f09f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-ofjduh_h/wheels/f5/21/2c/cfce6eddcf1239d9f0cf1480a57bbbfd53c118360d7d309a6d
Successfully built twilio


In [ ]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-3z4besfe
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-3z4besfe
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=8aa3ad55c52892131cf847fcc562a893e67c1c05df4fb7c170d7777f922ab6b1
  Stored in directory: /tmp/pip-ephem-wheel-cache-se6yl_90/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dep

In [ ]:
!pip install git+https://github.com/openai/{openai-python,tiktoken}.git

  Cloning https://github.com/openai/openai-python.git to /tmp/pip-req-build-ravpdpth
  Running command git clone --filter=blob:none --quiet https://github.com/openai/openai-python.git /tmp/pip-req-build-ravpdpth
  Resolved https://github.com/openai/openai-python.git to commit 17ac6779958b2b74999c634c4ea4c7b74906027a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/openai/tiktoken.git to /tmp/pip-req-build-whvgeo56
  Running command git clone --filter=blob:none --quiet https://github.com/openai/tiktoken.git /tmp/pip-req-build-whvgeo56
  Resolved https://github.com/openai/tiktoken.git to commit 39f29cecdb6fc38d9a3434e5dd15e4de58cf3c80
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import runtime, userdata

In [19]:
import torch
import time
import whisper
import openai

LANGUAGE_ISO_CODE = "en" # @param {type:"string"}
whisper_model = "base.en" # @param ['tiny.en', 'tiny', 'base.en', 'base', 'small.en', 'small', 'medium.en', 'medium', 'large-v1', 'large-v2', 'large-v3', 'large']
MAX_RETRIES = 10 # @param {type:"slider", min:1, max:21, step:1}
TEXT_MODEL = "gpt-4-1106-preview" # @param ["gpt-4-1106-preview", "gpt-4", "gpt-3.5-turbo"]

openai_api_key = userdata.get('OPENAI_API_KEY').strip()
openai_client = openai.OpenAI(api_key=openai_api_key)

def load_model() -> whisper.Whisper:
  for i in range(MAX_RETRIES):
    try:
      model = whisper.load_model(whisper_model, device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
      i = i + 1
      print(f"Successfully loaded Whisper on {i}{'st' if i == 1 else 'nd' if i == 2 else 'rd' if i == 3 else 'th'} try")
      break
    except Exception as e:
      print(f"Error loading {whisper_model}: {e}")
      time.sleep(1)
      print(f"Trying again ({i+2}/{MAX_RETRIES})...")
  else:
    raise Exception(f"Failed to load Whisper model {whisper_model} in {MAX_RETRIES}, giving up")
  return model

# Allow the notebook to be run more than once. Fixes: runs out of memory.
try:
  model # this will throw an exception on the first run
except NameError:
  model = load_model()

In [41]:
import os
import tempfile
import openai

def create_named_pipe(pipe=True) -> str:
    # Generate a unique path for the named pipe in the system's temporary directory
    temp_path = tempfile.mktemp()

    # Create the named pipe
    if pipe:
      os.mkfifo(temp_path)

    return temp_path

def audio_in(named_pipe_output: str) -> None:
  """
  Get the audio from Trillio

  TODO: actually get it from Trillio
  """
  import base64
  print(f"[audio_in] Note: Trillio not implemented, using Test data only")
  HELLO_MP3_BASE64 = """SUQzBAAAAAAAIlRTU0UAAAAOAAADTGF2ZjYwLjMuMTAwAAAAAAAAAAAAAAD/83DAAAAAAAAAAAAASW5mbwAAAA8AAAA9AAAZnAALDw8TFxcbGx8jIycnKy8vMzM4PDxARERISExQUFRUWFxcYGBkaWltcXF1dXl9fYGBhYmJjY2RlZWamp6ioqaqqq6usra2urq+wsLGxsrPz9PX19vb3+Pj5+fr7+/z8/f7+/8AAAAATGF2YzYwLjMuAAAAAAAAAAAAAAAAJAKAAAAAAAAAGZyB2vNEAAAAAAAAAAAAAAAAAP/zQMQAELG2QAAL1JVf98d73pRsOQWwvCfTCUORkVlY+IDzUN/e97+HHpTRhn/r0MbM///9kMQVCUSAExADQwVBEEhhIeHh4eHgAAAcAzw9dbeYCnEJv2Honli9vBApT3VTdiyg+LhG//NCxBgYGuqOXkFFoPE2DQFALBEPSvxQUQyP793Fxzwn/v8IiDnM+SDJd0RP/O6FFwgRQMWLJ//IynRqEZf/zuhFOcDNe3//5Fsp0QQocDFvDwY/8opyyxtKVRu20a0joBHFAZUlJXUx//NAxBMY8ubGXjrKlsYpgacTOZUIrTf/8oemR+/0ibKMU/8px5BAoCgmqFuLhwxyX/qZFOHQ+ADjgYgpZ//FFMTYTs7HpV9ibyP6I3//9hIqiQULC6gGOKAMIo/+QnJLKfhYTjg1VMT/80LEChZq4r4+asTawUGVGYThiDFiQ7VyFIg7TZNuE7xAk+Jvyo4ihVfHL1bpbZ//1KTHmhJPymx+w7a6C3/10qcWpzp//2W9Gd//orYlsn///1lc7oENQpn/n0X/eSQoAFmEMAw44pD/80DEDBc5qro8ywSQ2hlbQCGXjAIKfaaam671h3HtQGAFBHjlgcB0KAjiWsMKhOO5kYUn86TCBANduc7MT+yh2IWjITaf1/r6MogMBkQKKILfZiBTjNv///0oB4tauqiW7KdCJAwCN//zQsQKFxiW0v7esgQYCCHJjQQpREAfBrxsIxvwC7BqV/yqaBylppjRALBAZa2MiYNweuOplySUPDQsjd6G2/ovtFkC4ezDR7Xa5EgMUg+gBizkoAaYIoOo/////+vC9Q4CHnoSWKpi8v/zQMQJFeGewDR4jSiLC6YlVBP86mJIqglCfPw7UqcibUbgtnWUxjoBRE7KRQJOW+tbMp0z2dGpT0rkUG24PzrhDfJiL0913uzDiwgwsHPeUxAT//////YZ//6VALAR3B0j9CGBlogi//NCxAwV2uLUV0k4Al6XJmzcRG0WQwM2wfSICSFhALwwJJxjdj0c5KGHGOyvf/a1PdHQ1mU9lqde1FmqYgtLjysv/////////2VumqMfnnISjxD/ZQ0cOcpuNyqRyJ7bXRxugDbABgys//NAxBAX0bMSX4xAAqENeq0zvHJiFjj6xaAqDUBUsUFzNQHAvCFyKGjR4ocLKAFKLkAIKI6r/V8p7veqsUPbmIdGiKd3M2lUX+Yido8UQ8XF6Jf///lX/iA4R+UqguBECcHEoyUHKXj/80LECxdhesArj3gAE2bT1LiilAaIjppD8cTmLcW4SY/EmpJn8cmIxVMW4YiHq85k9XFuqbarilPXX9d/OM//+1oUF7StZ3kBdubP6UeXa86r8Qq6/gCUg62QX///+Roj4cNY7TTIWhD/80DECRZhVrQpj3gAUwxWUry4JKEPkTRWLgvwhQuLiTJLqRQNrLFLaCyKkTcW8x0SjC5QNwbIRE9K2t/nFodPSmv/iExR1bKcqS1R5EQ9D2dkgvXvwxVfHjxGGKaKI/PJ2T4bqVQ4Zv/zQsQKFrFasCmPeACSRSIUrjbL8TMf8MkJhj4P9JKhneNxClWlBwGeLGih6jqG6LELQeMaJTXxhQvpffGMx//61x7Sv25QKRoUbRWskBXPqLlcH+pzQNN7HxH0+GaDHHAD3sD8uIoBpP/zQMQLFwlOsZGPeAB+2XBCRMDFepwuZ+mmSkkBprSvOdLK3RvKN/HQ9hVDKIUW06FixyQNRFTBe9DHmr09IE1a42pb7o8miZvLvG8q1sTqSV8ajyIzwogOO4lnacpVmWqaMmRPdUZY//NCxAkWmXrKWY9gAf+KYKUW8RtsMoXOO5pxDu2EGVjg2obGcnunZ+Vj5d4clBYsczzkCK05aP196HTzXTMyzSaLHJyZS2trRk3ZevfMDyJy1ray7Kq07szsueovoSowxN4A3irEnMse//NAxAoWiaKkAY94AIOowixqgWwsAjT2SIq5InrW1Oz3+Xqtprt6fVagxluq8eRHPW1YnFRi0JXTwIkJWMiF7ory3o86N1rVOmcX1nn0zn+hYsET7///cdf//9kiVWCzCTtbsUAEAIn/80LEChcRorZZj2AA9ML5z1muZyPXZyq0I0rjTZxClVAdEoGBXJ8D04uXLTN/IVuHRlq9fd3WTHOtq0xv7Zmr9nVvTM1KpqcuX9srlsRJmusrCSY1mZmZksdMzMzJF5WhCD/8FCAwxbz/80DECRZ5nqmJj1gAYqRSZpkvEnWZY5K2BfupB61WBkveYIplpfSqRx9Z9jIJBMH/SNpZTyQgcUDwPh1JqJqceOw5l8aA7B0AH/CQIQB1Mq1QQwRwDB/q//2t//yYyCRu11BQRQGxgv/zQsQKFsmirbmPYABkr6jyoWFEzl/Q0mJnOy3uZ6OVmVo+q1jrK3eaVrIt62NEqtG15MyxWrktGR8iXnjZ/ktGRlM7CDURBFcmb75YmZ1kBJ1x1PzPzND9z/n+k+tVgPvwCQD6BEtvuP/zQMQKFrGCiPGYeABn/gcKjYi0nKuWfZuXuc9+dI1C8QGFO292eSJWDPbdYFY5OMPifMP/5LJh6BwOEBXSK5m1iRXPsau3lvLmCc/r2Etu///+TiLjP//dsWEXEa2GDP/8k4us8iVG//NCxAoRAP5ABckYAEcJlqRFXNNJM4KW1onPtAQo0anrV+BhQIiDOonwYq7qesSlQV6q08U/p1eq4m06ISTv79KP/dWgnSsqSAz1V+60iebMBomRIWk3IXH1YmbKTVihS3PRH9ZmM72F//NAxCIPMPYwDDpENADq7euhtiqiihT/T/nnrdElH6lOxTY7ot78XSACCtu/JKHICgZDk+edSIUiU4LKP2itfCpXNWMQdOzy04BOaBg0Am3n3vT0zNyROdsQmxH9zOgkHW2mLb2pRXT/80LEQBGg6iFIaYY4Mb6Vtv1Y9UAaRgEuOfiYNWBo6dFYs9IsFAor+g8ixfWiZME2NKIMkWgBT6OsqrX97OpuOr7xopPaO7rc0lnjajQAIWqqA5tPwUrWjJ6Du9gtVTtnVdgxsfap52v/80DEVQ4YPiS0UEYIQ8qgWXalKOzNCldi13UkzOq9/F0J33u9PVkKBSs0sBZ9hZJp5FIjSsZsVVODPOupFHNeyZJvYq6yQImjlCBKp2imISDnMUKPO3DeL03LAlzH7XX9kVTQE0oqBP/zQsR3DaAGKZRQhACb3MMoKh4w523WQv3fEgumQBBHVsVX4cHkTa+3Nvsc2o23r55095GP0NdG7f7L6fMYgFgwQADAvGQhCM0t/4f/qr2P/r3svc1tnpoS3Za36EOdGIcjaudgYFAZnf/zQMScD5AGJNRYRADqiYf0od4OcvaLMvDeztedz/6Bx4JAkHsEMNwKANjYO9ZImSyZv3s2fxX/vezJ5IAgNB0gcAHAYEEIrC+j/g/VlIACwtiCk+YAzyUBadtF8ww+4eoj3XYLgmBY//NCxLgS4aYkAFMUePhMUEg/28////65PqAACp0nKk8/Nt/0//1V6Bzsrtr///////fe3///QmgAwdwAQRPgKrttrcE1phsA7D8I6HigOQwAzaPyG0cYu4lZLHQeTaQT679TC5+xQGOo//NAxMgUCa5kWCvXBIHVft2/+hGu7SN8l3/T/oSQhCXV3clv//////9CE7vU7/3VtlO5GQ6lS7CHRpNavhsAKzgsP+wI8LIoMMSC9QN6jQF0jPokWdUDm2ft91MyzCq0B89pD74re8r/80LE0hNK5ngAMkUNR2XvW+f7+P38I3kC/lW/r1X8H3545Torp+qhOPbD5iOgmlpQsYyFb//////+7MFdVLe16MdSpo7sDLY6gjlYegnp5tcAqIBUkjHj4gB+Kx0haBpMseEVvqvPHlL/80DE4BYC+spewkSNumzaK/tQ5+ToX/hKc8gkydIKOdW5g2gS5dGQwBDlAUSaTr2047GvQXUodBCh0IDilIiio4G/9vq137hzQqTaEFnFwi0wG3CodxaNVuZESkxcgkBgCDACL820YP/zQsTjG/r6sZTSxPkhw9zpiMiDTpmyHYzwOKwF1S74cDMGjSev3YbjVpwGIM2lbkMNh2G3baagIBCihjTeubthpudS9rRvULsIwqqqTG20C0Lln268r2as1InkJtVgRohuIoRmXYBJ///zQMTPGmmWrXbLCq7/0rt9E/f/u96JdrZu+eRNSshfS76/b5y9EaYY1QAgI5AH0KiwqnumVAAzKmNRCnOL9HDmbL5UY4UJcPGoozeGIZbg/jCQURvxAb1qRmpa1jCiDcGDCQ/fjh3+//NCxMAgW2KZjtpFFo6bSezMo9fyXtj0f/Oytl7u7uUms9R7ID62g31wqj8cD2ZRnVRYaxJnyACE///1r62r//5beVZj1PdVKYiPSEIKYyj/QHQsMWQHY5I2lfJRjZZmta/H2Lfuil2a//NAxJohctKRhtsFFOwBhtNlZM8ystwXhf1KgcBdWPMhViVwoo3qKiEh1WuORVz7YuEETjC8yPGb3mGuKT6aDBCFDJ3GxmwzvHjwYyDUTRxOFHmMcJYCYwKiWRTEY9t1tIzzD1f/////80LEbyGrYtpeyZUz//72sx7ozoSEB0pk7ve///1ZzjmOITRqRjTackSM4gH6T30mWypA1Vs7xxLSTCDOR3o2uTis3p5mhnuoYY41ToXN1K0f31eN851l5Mnl8sDIAieBgQiZroWGA0D/80DERBZ63vJeeEVPpkFi2f//zhGM1v//////9eV7KIcEgUYUCwLiAfI03eRO1TUmeIY0Wm5gxslmzhwjku9orR4OAywcAZZK40MuF+tF+/mfd2RJMR674WkupO4NpNKbgVpCQA0SXf/zQsRFFDGCzZZ7EKj8opn///8DipSOASGEkEkCQAXigasLitrgUmzwpKMJLoSGIVO8fVjcEKdDjNhg2zMtjhwRi3IY4px5BEhEww4hjRj1otHoz2ul9smWJDFfpFirjqP//+pWoNm0kf/zQMRQFAGW1l54xLgAFyRtIPrAJwveQAy8C7rQGnhqOFROKfC5RUaRZtynynzKtq9a05Pecd0EzxyjLHNFlJv/+NAUSFRBQ0VKjTqS1X4Vnv/9xSFp6FHih6InrckD+WWAHrQP13Qz//NCxFsUQWbaVmGKmgxrk0umS5sZp1vP7axP4fljctGikjaGE4Wlrf/0ECDiRg0pnbmduRSGixydgYMBBRnLj9H4RtSAnT0l8tsxxrUeTYee5FjFAAAklklskQQlkjbAD6FumEDIXSQ0//NAxGYUIV7Nt08YAi38BFmINLLtBFxPMmM15DtFWZRGVDZXi6RaGPNxjdUMyhQ5fiMr2JRSvY75Qq7FD9KppZ8L79CV60FCUssSruGq0czNLDAU0zE5n8oVc2qVSH+kSiO5YdVpr4X/80LEcCWZ/rpXmHgCWodQa51+di7g03b/33mmN6//zekmnhrCE0PxQZ5YhPv5VDEP4hNqRJLttstkqFBXgAkACq5dZRtJ2WqPAwAnWIwMSCkoeSoSGOkRFdAOmxcknw6FoDgiUosOC+//80DENSEhrspXmmACGQ4Fg7bsuPDuFcBQRDKd9jKFRQvSCcBMKxekkQK3RL6PLGywYUrRYwWzCx8sLg5CxDP9/pmZF5+5szMyOIjrHngh39///+qTuUuJiQbx0Vooo+WNAgT60fKeYv/zQsQLFHla1l/PKAI0jVXiEso6zxlWPSKerf43btN1e/qPU1Nq1yUYZnjWQTiZREEgAUaOKOl/RorMKwyZT9Bh/+Cwl//EJUQmAqmz9rUbTDf00wABP8AcWQepkyne6SBFRxPZZz6peP/zQMQVF9lmvjdPSAKQVuH/DmbJ70gg2Zm24QjNEpfr3G1vaJIUpsDxs9hclQTpWMP//uxjLGA0bGxSkzEmax7+hh7///0Q1Onq9RSoH2Da1M27cbDpgeotAJyO8MsGaAspz9dGqf4z//NCxBAYiabCeY9IAdSjaUZJ3Eg5kpAXli70ObSwqEKxUNoydGqYNKvQJjLayMUEqxJhIgKERAiZErUVxcMT0oZEBzI5VTFCl1kIvFH9a0TTLqff/+BhG6f2vwmamepVAAO/6qCh/SAl//NAxAkWQaKtkY9gAJ+IRV0KaJkdoTCkMY4dG7CMrX4dc8ef2MntYl6zKOWQxJXMFdo6Vr7sLhpPltUMzM82ggmDlMs0y6NR7s7rDiz5nmGmCU9MzMzKe1pmZmWOQAku8xO/bCUJVA//80LECxSBRsY/zCgCJsTIkio6ChUarqNTs7MOs6aZB6AcwmUhUYyGlr2UcZ3ZQVRyve9/5FYREQJgiAflSTf/KXf/o6n30CAWWDiBQEVmE895BBNkF6NSuOf/Yc4PljZa6PEQnkDPZRH/80DEFRIhWs5eSMS2CTQkVOFO1XIDCULhcBcTrs+7GKSDsY6vf/6bHKDRGiSTfgsAiJP/o//0k6PFhiK1tb/5FSAXI04q3hsBMo5VYdOKheVj8Kj+McQK1zGjcTfGmnSTHWj55ffPvf/zQsQnE8lewl5hhJ6/7WdExgI8H/G+3W/LLAZhkGYZCvSEyf////pce2m0tCJcQgY2MFkWKmgFa3M7Lx8BrYicHJ5qVeUnoUh0lYWXWvTZT0xBKxGmq9eR5qO5uaXc6pKLIz39v0xZBv/zQMQzE0Fazl5hil6RgKPBwGwn4iGEn////UAhkufAZdK1JidFzEAqbbSwtA/GU49Ii2KhuPC1BSbl5kzsZAcshUYittVqrto6raORFRxd1//2vs7h3sc/opLN6SEK9G/////9G2ch//NAxEETcqq5nnjEcxDnUxXCDSdpOgHc1/aELVpSSR73AQUyewpTrwjw53CBLB42DRx2I+gHVF1lfqCFEBGbPNCsJe3/MtiHHZXY+f7sqzO0WUxR/+/+t0oy2T////+v9CLGpIx2dBL/80LEThSCws5eeMS7X8kU20EbrJHG3RKBmAvjB1z0CBzZXJtLmYgSOd1LH1C4l5ocjL7A5+0MfxlRMpz6Ce+1DHqUbT//Z6DlKAipQQKmJT2Na07//+68clqAqBg9UKMXQBlacsjd1oH/80DEWBP5dsJeewSuuJCI1kkU5oUJBaJooNy21mvdbFbYWWEzUVWbqUr+3BETQiisIhMIlZP//NWaExQ/w//IyYcGev//5lhL+eYvCUM4Kk1OmRQBE4jYbdKBrSZslqIVioGVDRADyf/zQsRjE4K+sl5IR1q6QI6ZWAshI3UD2tVXpX8NFgoBFMFTVCn5pUV1DxEweRPv6fMt/tbx2Of3a6Q7o7f/UhQAACo04BvBVbjCYiYNnUKzaSCYGYDILCeJH3y62Jjf9qnfhku15p1Onf/zQMRxEajuRApJhsgHfx/K3U2r3TzJU1X7F+KFXtCVIACC7jYG9Ltqw7V3hbhguha4CGtA8ndFqG3/2Vf6UIJf7v92nay17k9mtW27+1sUQgGruqq2aBkyQCTOJELeiQVSLF5Uy4Bi//NCxIUPKGZJ/hBMAM7e9bvuVlbK4NkHWDSdSxEbi36B9byViiqESrFM82p8kAXiXR9dgbUZcLozo/il1ouqHAAI/g1Yd/GkuzCQqNJIUpb3AZji7G0VBJRVh7QxS1EnHphYlg6eUECJ//NAxKQMAFpFdhDEKEG0/uxpdBh74r0sQAxrU8XLR7hr9Rpakj75CgZTbNjbuWA6QNVOiHlSCK1M1pVeygUhmTi8znzUQl9NJMS/0rHqmpZc4nkQq3sx2fSJ2vNf/tSgVBfwzMSrfyj/80LEzxJI9hyiaYQ8yMs3Inbl4l1eDYfx2SR0zvGuSuCLXSffMwoIn+Zx8qE2QtpBSIMltCj1Ww9niVH+2fFNy9SdPSne4OICKhAsB+s7GjvFHeMJu+ayo4Sr6wJHcPcZcVTH23wvGhL/80DE4REgTh2KaEYE9fIOjAjO03Y8TIzRTt9tCHOUr5M+UUH7k2hu21zVYVhXuvk7TKyVyInNTqCbckp5ltsR86uRvqS5KyUo8itZCuUgmY5ImOppYggjCFiZrvXomMv7HwNgMkaRyv/zQsT3HxNh/ACYTAXZ2UWOrualdruJIKbBYpIKbFxf+Fk/XnDRX/cb/ExU0kN/f9+LC//lFcOCouSVsUFReP7v/7l+TZPn45Sxf6adw0K/E0V/8kxBTUUzLjEwMKqqqqqqqqqqqqqqqv/zQMTWGfMmCMh6RhWqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq//NCxMkWMJHwwHjMLaqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq"""
  decoded_bytes = base64.b64decode(HELLO_MP3_BASE64)
  with open(named_pipe_output, "wb") as f:
    f.write(decoded_bytes)

def stt(named_pipe_input: str, named_pipe_output: str) -> None:
  """
  TODO: streaming
  """
  print("[stt]")
  no_speech_threshold = 0.6 # @param {type:"slider", min:0, max:1, step:0.01, default:0.6}
  logprob_threshold=-1 # @param {type:"slider", min:-1, max:1, step:0.01, default:-1}
  transcript = model.transcribe(
      audio=named_pipe_input,
      verbose=True,
      no_speech_threshold=no_speech_threshold,
      logprob_threshold=logprob_threshold,
      language=LANGUAGE_ISO_CODE,
      without_timestamps=True,
  )
  with open(named_pipe_output, "w") as f:
    f.write(transcript["text"])

def reason_and_reply(named_pipe_input: str, named_pipe_output: str) -> None:
  """
  TODO: streaming
  TODO: agents API
  """
  with open(named_pipe_input) as f:
    query = f.read()
  completion = openai_client.chat.completions.create(
      model=TEXT_MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant"},
          {"role": "user", "content": query.strip()},
      ],
  )
  with open(named_pipe_output, "w") as f:
    f.write(completion.choices[0].message.content.strip())

def tts(named_pipe_input: str, named_pipe_output_mp3: str) -> None:
  """
  TODO: input streaming
  TODO: change output from mp3 to Opus (lower latency)
  """
  with open(named_pipe_input) as f:
    input_text = f.read()
    response = openai_client.audio.speech.create(
      model="tts-1",
      voice="echo",
      input=input_text,
    )
  response.stream_to_file(named_pipe_output_mp3)

def audio_out(named_pipe_input_mp3: str) -> None:
  """
  Send audio back out to Trillio
  """
  print(f"[audio_out] Note: Trillio out not implemented, using IPython widget instead")
  from IPython.display import Audio
  with open(named_pipe_input_mp3, "rb") as f:
    with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as temp_file:
      temp_file.write(f.read())
      temp_file_path = temp_file.name
  # Show a playable IPython widget
  print(temp_file_path)
  display(Audio(temp_file_path, autoplay=True))

def set_up_call() -> None:
  """
  TODO: handle hanging up
  """
  import threading

  audio_in_pipe = create_named_pipe()
  text_in_pipe = create_named_pipe()
  text_out_pipe = create_named_pipe()
  audio_out_pipe = create_named_pipe()

  # Creating threads for each method
  threads = [
      threading.Thread(target=audio_in, args=(audio_in_pipe,)),
      threading.Thread(target=stt, args=(audio_in_pipe, text_in_pipe)),
      threading.Thread(target=reason_and_reply, args=(text_in_pipe, text_out_pipe)),
      threading.Thread(target=tts, args=(text_out_pipe, audio_out_pipe)),
      threading.Thread(target=audio_out, args=(audio_out_pipe,))
  ]

  # Starting all threads
  for thread in threads:
      thread.start()

  # Optional: Joining all threads
  for thread in threads:
      thread.join()

  print("All tasks completed. Cleaning up...")

  # Remove the pipes after use
  os.remove(audio_in_pipe)
  os.remove(text_in_pipe)
  os.remove(text_out_pipe)
  os.remove(audio_out_pipe)


time: 1.94 ms (started: 2023-11-30 08:50:00 +00:00)


In [42]:
import pdb

set_up_call()

[audio_in] Note: Trillio not implemented, using Test data only
[stt]
[audio_out] Note: Trillio out not implemented, using IPython widget instead
[00:00.000 --> 00:01.530]  Hi, who are you?
/tmp/tmplvd0jj0s.mp3


All tasks completed. Cleaning up...
time: 5.98 s (started: 2023-11-30 08:50:01 +00:00)
